## AI model to predict waste images.
The dataset is acquired from
https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification/data


In [1]:

import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as v2
from torchvision import transforms
import torch
import torchvision
from torch import nn, optim
from torchsummary import summary

In [5]:
root_folder = ""

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
root_folder = "/content/drive/MyDrive/AI Projects/Waste_Classification"

### Parameters

In [8]:
dataset_folder  = root_folder+ "/Dataset/images"
batch_size = 32
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

print (f"Settings ... \n dataset path \t\t {dataset_folder} \n batch_size \t\t {batch_size}" +
       f"\n train_ratio \t\t {train_ratio} \n validation ratio \t {valid_ratio} \n test ratio \t\t {test_ratio} \n device \t\t {device}")

Using device: cuda
Settings ... 
 dataset path 		 /content/drive/MyDrive/AI Projects/Waste_Classification/Dataset/images 
 batch_size 		 32
 train_ratio 		 0.7 
 validation ratio 	 0.15 
 test ratio 		 0.15 
 device 		 cuda


In [9]:
class_labels = [name for name in os.listdir(dataset_folder) if not name.startswith(".DS_Store")]
print(class_labels)
num_classes = len(class_labels)
print(num_classes)

['aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_packaging', 'clothing', 'eggshells', 'food_waste', 'newspaper', 'office_paper', 'disposable_plastic_cutlery', 'plastic_trash_bags', 'steel_food_cans', 'styrofoam_food_containers', 'plastic_food_containers', 'plastic_water_bottles', 'plastic_detergent_bottles', 'tea_bags', 'styrofoam_cups', 'plastic_soda_bottles', 'plastic_shopping_bags', 'plastic_straws', 'plastic_cup_lids', 'shoes', 'paper_cups', 'cardboard_boxes', 'aerosol_cans', 'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'magazines', 'coffee_grounds']
30


In [27]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

#transformations = v2.Compose([
    #v2.ToImage(),
    #v2.RandomHorizontalFlip(),
    #v2.RandomVerticalFlip(),
    #v2.RandomResizedCrop(size=(256, 256), antialias=True),
    #v2.RandomRotation(degrees = (0, 170)),
    #v2.ToDtype(dtype = torch.float32, scale = True),
    #v2.Resize(size = (224, 224), antialias = True),
    #v2.Tensor(),
    #v2.Normalize(mean, std)
#])

transformations = transforms.Compose([
                    #transforms.Resize((224,224)),
                    transforms.ToTensor(),
                    #transforms.Normalize(mean,std)

])


dataset = ImageFolder(dataset_folder,transform=transformations)

### Creating and splitting the dataset

In [28]:
train_size = int(len(dataset) * train_ratio)
valid_size = int(len(dataset) * valid_ratio)
test_szie = int(len(dataset) * test_ratio)

train_dataset, valid_dataset, test_dataset = random_split(dataset,[train_size, valid_size, test_szie])

train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset,shuffle=True,batch_size=batch_size)
test_dataloader = DataLoader(test_dataset,shuffle=True, batch_size=batch_size)

print(f"Lenght of training dataset {len(train_dataloader.dataset)}")
print(f"Lenght of validation dataset {len(valid_dataloader.dataset)}")
print(f"Lenght of test dataset {len(test_dataloader.dataset)}")


Lenght of training dataset 10500
Lenght of validation dataset 2250
Lenght of test dataset 2250


In [29]:
import time
start = time.time()
x,y = next(iter(train_dataloader))
print(x.shape)
print(y.shape)

end = time.time()
print(f" time {end-start} secs")

torch.Size([32, 3, 256, 256])
torch.Size([32])
 time 22.643102645874023 secs


## CNN model

In [ ]:
class CNN(nn.Module):
    def __init__ (self):
        super().__init__()
        self.network = nn.Sequential(

            nn.Conv2d(in_channels = 3, out_channels = 4, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #256,256,4
            nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #256,256,8
            nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #128,128,16
            nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #64,64,32
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #32,32,64
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #16,16,64
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #8,8,64
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #8,8,128
            nn.Flatten(),
            nn.Linear(128*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),

        )


    def forward(self, x):
            return self.network(x)

### ResNet

In [13]:
res_net = torchvision.models.resnet50(weights='IMAGENET1K_V1')
#res_net.parameters().requires_grad = False
res_net.fc  = torch.nn.Sequential(
    nn.Linear(2048,128),
    nn.ReLU(),
    nn.Linear(
        in_features=128,
        out_features=num_classes
    ),
)

res_net.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 184MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model = CNN()
model = model.to(device)    #converting model to gpu from cpu
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3 )
print("Compiled Successfully")

Compiled Successfully


In [ ]:
from torchsummary import summary
model = res_net
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(res_net.fc.parameters())

In [ ]:
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x,y = x.to(device), y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            correct += (predictions == y).sum().item()
            total += predictions.size(0)
    model.train()
    return correct/total

In [15]:


def train_model(model,criterion,optimier, data_loader,device, num_epochs=0):
    model.to(device)
    for epoch in range(num_epochs):
        loss_batches = 0
        corrects_batches = 0
        for x,y in data_loader:

            x,y = x.to(device), y.to(device)
            print(".",end='')
            outputs = model(x)
            loss = criterion(outputs,y)
            optimier.zero_grad()
            loss.backward()
            optimier.step()
            print("*",end="")
            _,preds = torch.max(outputs,1)
            loss_batches += loss.item() * x.size(0)
            corrects_batches += torch.sum(preds == y.data)
        epoch_loss = loss_batches / len(data_loader)
        epoch_acc = corrects_batches.double() / len(data_loader)
        print(f"\n epoch {epoch} Loss : {epoch_loss:.4f} Accuracy {epoch_acc:.2f}")

    return model




torch.Size([32, 3, 224, 224])
torch.Size([32])
 time 23.38536787033081 secs


In [ ]:
train_model(res_net,criterion=criterion, optimier=optimizer,
            data_loader=train_dataloader, device=device, num_epochs=3)

torch.save(res_net.state_dict(),'resNet.h5')


.*.*.*.*.*.*.*.*.*.*.*.*.*.*.*.

KeyboardInterrupt: 

In [ ]:

print(f"Training Accuracy is {calculate_accuracy(train_dataloader, model)*100}")

print(f"Testing Accuracy is {calculate_accuracy(test_dataloader, model)*100}")